In [28]:
import random
from math import gcd

Написати функцію пошуку випадкового простого числа з заданого інтервалу або
заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на
простоту. В якості датчика випадкових чисел використовуйте вбудований генератор
псевдовипадкових чисел вашої мови програмування. В якості тесту перевірки на простоту
рекомендовано використовувати тест Міллера-Рабіна із попередніми пробними
діленнями. Тести необхідно реалізовувати власноруч, використання готових реалізацій
тестів не дозволяється.

In [29]:
def num_check(n: int, rounds:int=10) -> bool:
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    d = n - 1
    r = 0
    while d % 2 == 0:
        d //= 2
        r += 1

    for _ in range(rounds):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)

        if x in (1, n - 1):
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

In [30]:
def num_generator(length: int, start:int=0, end:int=0) -> int:
    if start == 0 and end == 0:
        start = 10**(length - 1)
        end = 10**length - 1   

    while True:
        num = random.randint(start, end)
        
        if num_check(num):
            return num
        
def num_generator_bit_length(length:int) -> int:
    start = 2**(length - 1)
    end = 2**length - 1

    return num_generator(length, start, end)

За допомогою цієї функції згенерувати дві пари простих чисел
p, q
і
1 1 p , q
довжини щонайменше 256 біт. При цьому пари чисел беруться так, щоб
pq  p1q1
; p і q –
прості числа для побудови ключів абонента А,
1 p
і
q1
– абонента B. 

In [31]:
class Pair:
    def __init__(self, p, q, p1, q1):
        self.p = p
        self.p1 = p1
        self.q = q
        self.q1 = q1

    def __str__(self):
        return f"p = {self.p} \nq = {self.q} \np1 = {self.p1} \nq1 = {self.q1}"

In [32]:
def make_pairs() -> Pair:
    while True:
        p = num_generator_bit_length(256)
        p1 = num_generator_bit_length(256)
        q = num_generator_bit_length(256)
        q1 = num_generator_bit_length(256)

        if p * q <= p1 * q1:
            return Pair(p, q, p1, q1)
        
pair = make_pairs()
print(pair)



p = 76399827603001855524334121892751927306361706179428662509452410845968380801223 
q = 86460645852046608844112519937426930154596747795381159617162966489146528554253 
p1 = 78857108443716013981620659492608550246372701023665740350258472967005070076493 
q1 = 108899931041722841988845073775605140775968644716883203649842931498179532090633


3.

In [33]:
def mod_inverse(a, m):
    if gcd(a, m) != 1:
        raise ValueError("Обертати неможливо")

    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1


def generate_rsa_keypair(pair: Pair) -> tuple:
    n = pair.p * pair.q
    n1 = pair.p1 * pair.q1

    phi_n = (pair.p - 1) * (pair.q - 1)
    phi_n1 = (pair.p1 - 1) * (pair.q1 - 1)

    e = 65537
    """
    while True:
        e = random.randint(2, phi_n - 1) 
        if gcd(e, phi_n) == 1:  
            break

    while True:
        e1 = random.randint(2, phi_n1 - 1) 
        if gcd(e1, phi_n1) == 1:
            break
    """

    d = mod_inverse(e, phi_n)
    d1 = mod_inverse(e, phi_n1)

    public_key_A = (e, n)
    public_key_B = (e, n1)

    private_key_A = (d, pair.p, pair.q)
    private_key_B = (d1, pair.p1, pair.q1)

    return public_key_A, private_key_A, public_key_B, private_key_B

pair = make_pairs()
print(pair)

public_key_A, private_key_A, public_key_B, private_key_B = generate_rsa_keypair(pair)

print("\nАбонент A - відкритий ключ:", public_key_A)
print("Абонент A - секретний ключ:", private_key_A)
print("\nАбонент B - відкритий ключ:", public_key_B)
print("Абонент B - секретний ключ:", private_key_B)

p = 91868263241667762279082801572419574565893510599008451297589815206933756858709 
q = 90627825810026110930832066867727108838819994879174090497157451904636456139641 
p1 = 111670058585076977347946503084892895194936061557624075061154891611349980372361 
q1 = 111919832703290812963818134269512256352067813373430546802078992434134540180967

Абонент A - відкритий ключ: (65537, 8325820958535490659588370045535185060512680654585981361443136552249575335832508820584056905995332069122263336224662785579358653681733407498639236810983469)
Абонент A - секретний ключ: (3425125636862770094346125773191847725963690482144325243539808117326102821755273357564579329600447615863364618661547810873001379104992389314466902957844833, 91868263241667762279082801572419574565893510599008451297589815206933756858709, 90627825810026110930832066867727108838819994879174090497157451904636456139641)

Абонент B - відкритий ключ: (65537, 12498094274808499297865964171027619265435445234092035561692107097914827046428029116437029866

4.

In [34]:
def Encrypt(message, public_key):
    e, n = public_key
    cipher_text = [pow(ord(char), e, n) for char in message]
    return cipher_text

def Decrypt(cipher_text, private_key):
    d, p, q = private_key
    n = p * q
    decrypted_message = ''.join([chr(pow(char, d, n)) for char in cipher_text])
    return decrypted_message

def Sign(message, private_key):
    d, p, q = private_key
    n = p * q
    signature = [pow(ord(char), d, n) for char in message]
    return signature

def Verify(message, signature, public_key):
    e, n = public_key
    message_from_signature = ''.join([chr(pow(char, e, n)) for char in signature])
    return message_from_signature == message

def SendKey(sender_private_key, receiver_public_key, k):
    encrypted_k = Encrypt(str(k), receiver_public_key)
    signature = Sign(str(k), sender_private_key)

    return encrypted_k, signature

def ReceiveKey(private_key, sender_public_key, encrypted_k, signature):
    decrypted_k = Decrypt(encrypted_k, private_key)

    if Verify(decrypted_k, signature, sender_public_key):
        print("Підпис дійсний!")
    else:
        print("Підпис недійсний!")

    return decrypted_k

In [35]:
pair = make_pairs() 

public_key_A, private_key_A, public_key_B, private_key_B = generate_rsa_keypair(pair)

k = random.randint(1, public_key_B[1] - 1)
print(f"Випадковий ключ: {k}")

encrypted_k, signature = SendKey(private_key_A, public_key_B, k)


decrypted_k = ReceiveKey(private_key_B, public_key_A, encrypted_k, signature)

print(f"Розшифрований ключ: {decrypted_k}")



Випадковий ключ: 3917696559780703199659293780267375211971671219864090594499189159837556086204827977079698393145660573983247637036932790131345707499035286352081915350495135
Підпис дійсний!
Розшифрований ключ: 3917696559780703199659293780267375211971671219864090594499189159837556086204827977079698393145660573983247637036932790131345707499035286352081915350495135


In [36]:
message = "Test1"
print(f"Початкове повідомлення: {message}")

encrypted_message = Encrypt(message, public_key_B)
print(public_key_B)
print("Зашифроване повідомлення для абонента B:", encrypted_message)

print(private_key_B)
decrypted_message = Decrypt(encrypted_message, private_key_B)

print("Розшифроване повідомлення для абонента B:", decrypted_message)


Початкове повідомлення: Test1
(65537, 7131419816832572144160515673961437908815918034033815514056376353835248566020661166927344682814636250959392663263173292468555029714780087712209835106937527)
Зашифроване повідомлення для абонента B: [6725517830163790027998037772414970125362777714568425735184751965766741543475855079234972733335569548652417998765550219320722089479985108553524978065201378, 111847227075867646521460741737350749752704824069036516027008333372867780687433180972802900717446767573730561503052692994530421140204046419228212245956784, 5346339715406317670245772993604180565096103105270345951351413817192229277999442165514318960437111204743692178813382873525174918644594195020452960807067544, 838693998167472898609130183325396370809138386747802853797584145260522531057851029547141398391369862706704374237099844737110844769292231384537495838637894, 2810111885864982612871725577741090431613568129262493243043471718066041525085226942884218716628850450768073500538670558403237214524560993980625